In [1]:
from IPython import display
import argparse
import cv2
import gym
import imageio
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, sys, shutil
import torch
import omegaconf
import time
from typing import Dict, Optional, Tuple, Union, cast

from matplotlib.gridspec import GridSpec

import mbrl.env.cartpole_continuous as cartpole_env
import mbrl.env.reward_fns as reward_fns
import mbrl.env.termination_fns as termination_fns
import mbrl.models as models
import mbrl.planning as planning
import mbrl.util.common as common_util
import mbrl.util as util

import tactile_gym.rl_envs
from tactile_gym.sb3_helpers.params import import_parameters
from mbrl.util.plot_and_save_push_data import plot_and_save_push_plots

from pyvirtualdisplay import Display
from tactile_gym.sb3_helpers.params import import_parameters
from tactile_gym.rl_envs.nonprehensile_manipulation.object_push.object_push_env import get_states_from_obs

_display = Display(visible=False, size=(1400, 900))
_ = _display.start()
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


pybullet build time: Mar  8 2021 17:26:24


In [2]:
root_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/object_push_rl/model_based_models"
# model_name = "relative_ext_space_normal_cem_best"
model_name = "relative_randomised"
work_dir = os.path.join(root_dir, model_name)

env_kwargs_dir = os.path.join(work_dir, 'env_kwargs')
env_params = omegaconf.OmegaConf.load(env_kwargs_dir)

# edit eval env modes here
goals = [[0.0, 0.18]]
n_eval_episodes = len(goals)
env_params['goals'] = goals
# n_eval_episodes = 10
env_params["env_modes"]['eval_mode'] = True
env_params["env_modes"]['eval_num'] = n_eval_episodes

# Common reward function for evaluation
env_params["env_modes"]['terminated_early_penalty'] = -500
env_params["env_modes"]['reached_goal_reward'] = 100
env_params["env_modes"]['importance_obj_goal_pos'] = 1.0
env_params["env_modes"]['importance_obj_goal_orn'] = 1.0
env_params["env_modes"]['importance_tip_obj_orn'] = 1.0

# set limits and goals
TCP_lims = np.zeros(shape=(6, 2))
TCP_lims[0, 0], TCP_lims[0, 1] = -0.1, 0.45  # x lims
TCP_lims[1, 0], TCP_lims[1, 1] = -0.3, 0.3  # y lims
TCP_lims[2, 0], TCP_lims[2, 1] = -0.0, 0.0  # z lims
TCP_lims[3, 0], TCP_lims[3, 1] = -0.0, 0.0  # roll lims
TCP_lims[4, 0], TCP_lims[4, 1] = -0.0, 0.0  # pitch lims
TCP_lims[5, 0], TCP_lims[5, 1] = -180 * np.pi / 180, 180 * np.pi / 180  # yaw lims
env_params["env_modes"]['tcp_lims'] = TCP_lims.tolist()

# Create environment
seed = 0
torch.random.manual_seed(seed)
np.random.seed(seed)

handler_env_name = "pybulletgym___" + "object_push-v0"
handler = util.create_handler_from_str(handler_env_name)
eval_env = handler.make_env_from_str(handler_env_name, **env_params)
eval_env.seed(seed)
generator = torch.Generator(device=device)
generator.manual_seed(seed)
obs_shape = eval_env.observation_space.shape
act_shape = eval_env.action_space.shape

# Hyperparameters
num_particles = 20
lookhead_steps = 25

# Get cfg and agent cfg
config_file = 'cfg_dict'
config_dir = os.path.join(work_dir, config_file)
cfg = omegaconf.OmegaConf.load(config_dir)

agent_config_file = 'agent_cfg'
agent_config_dir = os.path.join(work_dir, agent_config_file)
agent_cfg = omegaconf.OmegaConf.load(agent_config_dir)

# model_num = 100
# model_path = os.path.join(saved_model_dir, "model_trial_{}".format(model_num))
model_path = os.path.join(work_dir, "best_model")
dynamics_model = common_util.create_one_dim_tr_model(cfg, obs_shape, act_shape, model_path)
model_env = models.ModelEnvPushing(eval_env, dynamics_model, termination_fn=None, reward_fn=None, generator=generator)
model = planning.create_trajectory_optim_agent_for_model(
    model_env,
    agent_cfg,
    num_particles=num_particles
)

argv[0]=
Loaded EGL 1.5 after reload.
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3090/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 495.29.05
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 495.29.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3090/PCIe/SSE2
ven = NVIDIA Corporation
ven = NVIDIA Corporation


/home/qt21590/anaconda3/envs/tactile_gym_mbrl/lib/python3.9/site-packages/hydra/utils.py:32: UserWarning: `OmegaConf.is_none()` is deprecated, see https://github.com/omry/omegaconf/issues/547
  if OmegaConf.is_none(config):


In [3]:
# obs = eval_env.reset()
# eval_env.make_goal(env_params['goals'][0])
# done, state = False, None
# episode_reward = 0.0
# episode_length = 0
# trial_pb_steps = 0.0

# # Get model prediction using a optimised plan
# plan = model.plan(obs, **{})
# plan = torch.from_numpy(plan[None, :]).to(device)
# total_rewards = model_env.evaluate_action_sequences(plan, obs, 20)
# print(total_rewards)

# print('Episode reward, ', episode_reward)
# print('Episode steps, ', episode_length)

tensor([-43.5977], device='cuda:0')
Episode reward,  0.0
Episode steps,  0


In [3]:
# Plot observations
def plot_data(model_obses, real_obses, env_frame):
    
    num_plots = model_obses.shape[-1]
    rows = 2
    columns = int(num_plots/2)
    # fig, axs = plt.subplots(rows, columns, figsize=(18, 7))
    fig = plt.figure(figsize=(18, 10))
    gs = GridSpec(3, 4, figure=fig)

    titles = ['tcp_x', 'tcp_y', 'tcp_orn_1', 'tcp_orn_2', 'contact_x', 'contact_y', 'contact_orn_1', 'contact_orn_2']

    def plot_model_obses(axs, data, avg=False):
        num_particles = data.shape[1]
        if avg:
            data = np.mean(data, axis=1,)
            axs.plot(data, 'r', alpha=0.1)
        else:
            for i in range(num_particles):
                axs.plot(data[:, i], 'r', alpha=0.1)

    for i in range(rows):
        for j in range(columns):
            axs = fig.add_subplot(gs[i, j])
            axs.plot(real_obses[:, i*4+j], 'b')
            plot_model_obses(axs, model_obses[:, :, i*4+j])

            # Settings for the plot
            plt.axes(axs)
            plt.title(titles[i*4+j])

    axs = fig.add_subplot(gs[-1, :])
    axs.imshow(env_frame)
    axs.get_xaxis().set_visible(False)
    axs.get_yaxis().set_visible(False)

    # Convert to image
    fig.canvas.draw()
    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    # img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR)

    plt.close(fig)    
    
    return img


def get_model_real_obses(obs, env, agent, model_env, lookahead, particles):

    env_dict = {"env": env}
    plan = agent.plan(obs, **{})
    model_obses, model_rewards, actions = util.common.rollout_model_push_env(
        model_env,
        obs,
        plan=plan,
        agent=None,
        num_samples=particles,
    )
    real_obses, real_rewards, _ = handler.rollout_env(
        cast(gym.wrappers.TimeLimit, env_dict),
        obs,
        lookahead,
        agent=None,
        plan=actions,
    )

    return model_obses, model_rewards, real_obses, real_rewards, actions

# lookahead = 25
# particles = 20
# obs = eval_env.reset()
#   # When using MPC, rollout model trajectories to see the controller actions
# plan = model.plan(obs, **{})
# model_obses, model_rewards, actions = util.common.rollout_model_push_env(
#     model_env,
#     obs,
#     plan=plan,
#     agent=None,
#     num_samples=particles,
# )

# # Then evaluate in the environment
# env_dict = {"env": eval_env}
# real_obses, real_rewards, _ = handler.rollout_env(
#     cast(gym.wrappers.TimeLimit, env_dict),
#     obs,
#     lookahead,
#     agent=None,
#     plan=actions,
# )

# frame = eval_env.render(mode="rgb_array")
# data = (model_obses, model_rewards, real_obses, real_rewards, actions)
# img = plot_data(data, frame)

In [27]:
# Plot and output as video
def get_predictions(obs, env, agent, model_env, lookahead, particles):

    data = get_model_real_obses(obs, env, agent, model_env, lookahead, particles)
    frame = eval_env.render(mode="rgb_array")
    return data, frame

# Start evaluation
obs = eval_env.reset()
eval_env.make_goal(env_params['goals'][0])
done, state = False, None
episode_reward = 0.0
episode_length = 0
graph_frames = []
model_obses_trial = []
real_obses_trial = []
obses_err_trial = []
env_frame_trial= []
record_frequency = 5

while not done:

    if episode_length % record_frequency == 0:

        # print("before", eval_env.termination(0))
        # pos_workframe, _ = eval_env.robot.arm.worldframe_to_workframe(
        #         eval_env.cur_obj_pos_worldframe, np.zeros(3)
        #     )
        # print("observation", pos_workframe)
        # print('dist', np.linalg.norm(eval_env.cur_obj_pos_worldframe - eval_env.goal_pos_worldframe))
        # print('target_id', eval_env.targ_traj_list_id)

        # Get data 
        data_step, env_frame_step = get_predictions(obs, eval_env, model, model_env, lookahead=lookhead_steps, particles=num_particles)
        model_obses, _, real_obses, _, _ = data_step
        model_obses_trial.append(model_obses)
        real_obses_trial.append(real_obses)

        # Record model errors
        if model_obses.shape[0] == real_obses.shape[0]:
            sq_err_obses = np.square(np.swapaxes(model_obses,0,1) - real_obses)
            # print(np.max(sq_err_obses, axis=(0, 1)))
            obses_err_trial.append(sq_err_obses)
            env_frame_trial.append(env_frame_step)

        # Append graph frame into a video array
        plot_img = plot_data(model_obses, real_obses, env_frame_step)
        graph_frames.append(plot_img)

        eval_env.get_step_data()
        # print("after", eval_env.termination(0))
        # pos_workframe, _ = eval_env.robot.arm.worldframe_to_workframe(
        #         eval_env.cur_obj_pos_worldframe, np.zeros(3)
        #     )
        # print("observation", pos_workframe)
        # print('dist', np.linalg.norm(eval_env.cur_obj_pos_worldframe - eval_env.goal_pos_worldframe))
        # print('target_id', eval_env.targ_traj_list_id)
        # print("...")

    action = model.act(obs, **{})
    obs, reward, done, _info = eval_env.step(action)

    episode_reward += reward
    episode_length += 1

    # if episode_length > 50:
    #     done = True

result_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/object_push_rl/results"
video_dir = os.path.join(result_dir, 'model_analysis')
os.makedirs(video_dir, exist_ok=True)
video_file = os.path.join(video_dir, 'model_prediction.mp4')
imageio.mimwrite(video_file, np.stack(graph_frames), fps=10)

print('Episode reward, ', episode_reward)
print('Episode steps, ', episode_length)

Episode reward,  -353.70490140928507
Episode steps,  709


In [39]:
from tactile_gym.sb3_helpers.rl_plot_utils import plot_error_band

# Plot observations
def plot_model_error(mse_err, max_err, min_err, total_obses_trial, env_frame, steps):
    
    num_plots = model_obses.shape[-1]
    rows = 2
    columns = int(num_plots/2)
    # fig, axs = plt.subplots(rows, columns, figsize=(18, 7))
    fig = plt.figure(figsize=(18, 10))
    gs = GridSpec(3, 4, figure=fig)

    titles = ['tcp_x', 'tcp_y', 'tcp_orn_1', 'tcp_orn_2', 'contact_x', 'contact_y', 'contact_orn_1', 'contact_orn_2']

    for i in range(rows):
        for j in range(columns):
            axs = fig.add_subplot(gs[i, j])
            
            plot_error_band(
                axs,
                np.arange(len(mse_err)),
                mse_err[:, i*4+j],
                max_err[:, i*4+j],
                min_err[:, i*4+j],
                title=titles[i],
                colour="r",
                error_band=True,
            )

            # Settings for the plot
            plt.axes(axs)
            plt.title(titles[i*4+j])
            plt.ylim([0.0, 1.0])

    # Plot total mse error
    axs = fig.add_subplot(gs[-1, 0:2])
    axs.plot(total_obses_trial, 'b')
    axs.set_title('total model error')
    axs.set_xlabel('time step')
    axs.set_xlim([0, steps])
    
    # Show environment frame
    axs = fig.add_subplot(gs[-1, 2:4])
    axs.imshow(env_frame)
    axs.get_xaxis().set_visible(False)
    axs.get_yaxis().set_visible(False)

    # Convert to image
    fig.canvas.draw()
    img = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    img  = img.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)    
    
    return img

# normalise the model errors of each dimension to [0, 1]
obses_err_trial = np.array(obses_err_trial)
obses_err_trial = obses_err_trial[:-1] # Skip the last few data points, since error is too big here
norm_factor = np.max(obses_err_trial, axis=(0,1,2))
obses_err_normalised = []
for i in range(len(norm_factor)):
    obses_err_normalised.append(obses_err_trial[:, :, :, i] / norm_factor[i])
obses_err_normalised = np.moveaxis(obses_err_normalised,0,-1)

# Plot model error of each dimension over time 
mse_obses_trial = []
max_obses_trial = []
min_obses_trial = []
total_obses_trial = []
graph_frames = []
for i in range(len(obses_err_normalised)):
    mse_obses = np.mean(obses_err_normalised[i], axis=0)
    max_obses = np.max(obses_err_normalised[i], axis=0)
    min_obses = np.min(obses_err_normalised[i], axis=0)
    mse_obses_trial.append(mse_obses)
    max_obses_trial.append(max_obses)
    min_obses_trial.append(min_obses)

    total_obses = np.sum(np.mean(mse_obses, axis=-1))
    total_obses_trial.append(total_obses)

    graph_frame = plot_model_error(mse_obses, max_obses, min_obses, total_obses_trial, env_frame_trial[i], len(obses_err_normalised))
    graph_frames.append(graph_frame)

result_dir = r"/home/qt21590/Documents/Projects/tactile_gym_mbrl/object_push_rl/results"
video_dir = os.path.join(result_dir, 'model_analysis')
os.makedirs(video_dir, exist_ok=True)
video_file = os.path.join(video_dir, 'model_prediction_normalised.mp4')
imageio.mimwrite(video_file, np.stack(graph_frames), fps=10)


In [52]:
# eval_env.get_step_data()
print(np.linalg.norm(eval_env.get_observation()[4:6] - np.array(goals[0])))
print(eval_env.xyz_obj_dist_to_goal())
print(eval_env.goal_pos_worldframe)

pos_worldframe, _ = eval_env.robot.arm.workframe_to_worldframe(
                eval_env.goal_pos_workframe, np.zeros(3)
            )
print(pos_worldframe)
print(eval_env.goal_pos_workframe)
print(goals)


print(eval_env.get_observation()[4:6])

(
    cur_obj_pos_worldframe,
    _,
) = eval_env.get_obj_pos_worldframe()
print(np.linalg.norm(eval_env.cur_obj_pos_worldframe - eval_env.goal_pos_worldframe))

print("pos", eval_env.cur_obj_pos_worldframe)
print("pos", cur_obj_pos_worldframe)


# pos_workframe, _ = eval_env.robot.arm.worldframe_to_workframe(
#                 cur_obj_pos_worldframe, np.zeros(3)
#             )
# print(pos_workframe)
# # print(cur_obj_pos_worldframe)
# print(np.linalg.norm(pos_workframe[0:2] - np.array(goals[0])))


0.04060813198599847
0.040608132707686755
(0.7300000190734863, -0.15000000596046448, 0.03999999910593033)
[0.730000 -0.150000 0.040000]
[0.000000 0.180000 0.000000]
[[0.0, 0.18]]
[0.038288 0.166470]
0.040608132707686755
pos [0.716470 -0.111712 0.040001]
pos [0.716470 -0.111712 0.040001]
